# Feature Engineering
Objectives:
- Efficiently load and handle large-scale simulation datasets (spatial_all.parquet and timeseries.parquet) using Dask for scalable computation.
- Downsample and restructure the 3D reservoir grid into a computationally tractable form while preserving spatial integrity.
- Construct high-dimensional feature vectors for each grid cell, combining raw process variables and engineered metrics (~26 features configurable), to enable predictive modeling.
- Parameterize target selection (default: pressure [Pa]) for flexibility in supervised learning.
- Persist structured feature (X) and target (y) arrays in .npy format for downstream machine learning workflows.
- Maintain compatibility with batch-wise or Dask-aware data iterators to facilitate model training on large-scale datasets.

**Source:** Society of Petroleum Engineers (SPE)  
**Dataset:** SPE Comparative Solution Project - Model 11C (3D CO₂ Injection)

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
from pathlib import Path
import dask.dataframe as dd 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import os

# Paths
INTERIM = Path(r"C:\Users\tetec\Documents\Data Project Coding\.vscode\Project data\spe11c\data\interim\cleaned")
PROCESSED = Path(r"C:\Users\tetec\Documents\Data Project Coding\.vscode\Project data\spe11c\data\processed")
PROCESSED.mkdir(parents=True, exist_ok=True)

SPATIAL = INTERIM / 'spatial_all.parquet'
TIMESERIES = INTERIM / 'timeseries.parquet'
ML_DATA = PROCESSED / 'ml_data.npy' 
X_OUT = PROCESSED / 'X.npy'
Y_OUT = PROCESSED / 'y.npy'

 
NX, NY, NZ = 20, 20, 10 
TARGET_COL = 'pressure [Pa]' 
SPATIAL_COLS = [' x [m]', 'y [m]', 'z [m]', 'pressure [Pa]', 'saturation [-]', 'mCO2 [-]', 'mH2O [-]', 'rhoG [kg/m3]', 'rhoL [kg/m3]', 'tmCO2 [kg]', 'temp [°C]', 'year']
TS_AGG_COLS = ['p1 [Pa]', 'p2 [Pa]', 'mobA [kg]', 'immA [kg]', 'dissA [kg]', 'sealA [kg]']
TARGET_NUM_FEATURES = 26

In [ ]:
# Load data with Dask
spatial_ddf = dd.read_parquet(SPATIAL)
timeseries_ddf = dd.read_parquet(TIMESERIES)


print('Spatial partitions:', spatial_ddf.npartitions)
print('Timeseries partitions:', timeseries_ddf.npartitions)

Spatial partitions: 1
Timeseries partitions: 1


In [ ]:
# Select year snapshot
years = spatial_ddf['year'].unique().compute()
years

0        0
1     1000
2      100
3       10
4      150
5       15
6      200
7       20
8      250
9       25
10     300
11      30
12     350
13      35
14     400
15      40
16     450
17      45
18     500
19      50
20       5
21     600
22     700
23      75
24     800
25     900
Name: year, dtype: int64

In [ ]:
use_year = int(years.max()) 
spatial_year = spatial_ddf[spatial_ddf['year'] == use_year].compute()
print(spatial_year.shape)

(2016000, 12)


In [ ]:
x = spatial_year[' x [m]']
y = spatial_year['y [m]']
z = spatial_year['z [m]']

x_edges = np.linspace(x.min(), x.max(), NX+1)
y_edges = np.linspace(y.min(), y.max(), NY+1)
z_edges = np.linspace(z.min(), z.max(), NZ+1)

ix = np.digitize(x, x_edges) - 1
iy = np.digitize(y, y_edges) - 1
iz = np.digitize(z, z_edges) - 1

bin_df = pd.DataFrame({'ix': ix, 'iy': iy, 'iz': iz})
bin_df['orig_idx'] = spatial_year.index

bin_map = bin_df.groupby(['ix','iy','iz']).first().reset_index()
bin_map.head()

,ix,iy,iz,orig_idx
0,0,0,0,2016000
1,0,0,1,2217600
2,0,0,2,2419200
3,0,0,3,2620800
4,0,0,4,2822400


In [ ]:
# Extract representative cells
selected = spatial_year.loc[bin_map['orig_idx']].reset_index(drop=True)
selected.head()

,x [m],y [m],z [m],pressure [Pa],saturation [-],mCO2 [-],mH2O [-],rhoG [kg/m3],rhoL [kg/m3],tmCO2 [kg],temp [°C],year
0,25.0,25.0,5.0,28970000.0,0.0,0.000001,0.0,793.4,992.4,1.763,66.63,1000
1,25.0,25.0,125.0,32810000.0,0.0,0.000001,0.0,822.5,994.0,8034.000,66.92,1000
2,25.0,25.0,245.0,31540000.0,0.0,0.000001,0.0,826.1,995.2,8044.000,63.69,1000
3,25.0,25.0,365.0,30590000.0,0.0,0.000001,0.0,831.8,996.5,3.540,60.53,1000
4,25.0,25.0,485.0,29250000.0,0.0,0.000001,0.0,833.2,997.3,6448.000,57.81,1000


In [ ]:
# Base variables
base_cols = [
' x [m]', 'y [m]', 'z [m]', 'pressure [Pa]', 'saturation [-]',
'mCO2 [-]', 'mH2O [-]', 'rhoG [kg/m3]', 'rhoL [kg/m3]',
'tmCO2 [kg]', 'temp [°C]'
]

feats = selected[base_cols].copy()

In [ ]:
# Engineered features
feats['depth_norm'] = (feats['z [m]'] - feats['z [m]'].min()) / (feats['z [m]'].max() - feats['z [m]'].min())
feats['density_ratio'] = feats['rhoG [kg/m3]'] / (feats['rhoL [kg/m3]'] + 1e-6)
feats['co2_frac'] = feats['mCO2 [-]'] / (feats['tmCO2 [kg]'] + 1e-6)
feats['temp_dev'] = feats['temp [°C]'] - feats['temp [°C]'].mean()

In [ ]:
# Timeseries aggregates 
ts_df = timeseries_ddf.compute()
agg_cols = ['p1 [Pa]', 'p2 [Pa]', 'mobA [kg]', 'immA [kg]']

aggs = {}
for col in agg_cols:
    aggs[col + '_mean'] = ts_df[col].mean()
    aggs[col + '_max'] = ts_df[col].max()
    aggs[col + '_last'] = ts_df[col].iloc[-1]

agg_df = pd.DataFrame([aggs] * len(feats))
feats = pd.concat([feats, agg_df], axis=1)

feats.head()

,x [m],y [m],z [m],pressure [Pa],saturation [-],mCO2 [-],mH2O [-],rhoG [kg/m3],rhoL [kg/m3],tmCO2 [kg],...,p1 [Pa]_last,p2 [Pa]_mean,p2 [Pa]_max,p2 [Pa]_last,mobA [kg]_mean,mobA [kg]_max,mobA [kg]_last,immA [kg]_mean,immA [kg]_max,immA [kg]_last
0,25.0,25.0,5.0,28970000.0,0.0,0.000001,0.0,793.4,992.4,1.763,...,27700000.0,2.194050e+07,22170000.0,21940000.0,4.979564e+10,5.441000e+10,4.891000e+10,1.569244e+08,645500000.0,63390000.0
1,25.0,25.0,125.0,32810000.0,0.0,0.000001,0.0,822.5,994.0,8034.000,...,27700000.0,2.194050e+07,22170000.0,21940000.0,4.979564e+10,5.441000e+10,4.891000e+10,1.569244e+08,645500000.0,63390000.0
2,25.0,25.0,245.0,31540000.0,0.0,0.000001,0.0,826.1,995.2,8044.000,...,27700000.0,2.194050e+07,22170000.0,21940000.0,4.979564e+10,5.441000e+10,4.891000e+10,1.569244e+08,645500000.0,63390000.0
3,25.0,25.0,365.0,30590000.0,0.0,0.000001,0.0,831.8,996.5,3.540,...,27700000.0,2.194050e+07,22170000.0,21940000.0,4.979564e+10,5.441000e+10,4.891000e+10,1.569244e+08,645500000.0,63390000.0
4,25.0,25.0,485.0,29250000.0,0.0,0.000001,0.0,833.2,997.3,6448.000,...,27700000.0,2.194050e+07,22170000.0,21940000.0,4.979564e+10,5.441000e+10,4.891000e+10,1.569244e+08,645500000.0,63390000.0


In [ ]:
# Build 4D array (X) and target (y)
bin_map['ix'] = bin_map['ix'].clip(0, NX - 1)
bin_map['iy'] = bin_map['iy'].clip(0, NY - 1)
bin_map['iz'] = bin_map['iz'].clip(0, NZ - 1)

n_features = feats.shape[1]
X = np.zeros((NX, NY, NZ, n_features), dtype=np.float32)
Y = np.zeros((NX, NY, NZ), dtype=np.float32)

for i, row in bin_map.iterrows():
    ix, iy, iz = int(row['ix']), int(row['iy']), int(row['iz'])
    X[ix, iy, iz, :] = feats.iloc[i].values
    Y[ix, iy, iz] = selected.iloc[i][TARGET_COL]
    
X.shape, Y.shape

((20, 20, 10, 27), (20, 20, 10))

In [ ]:
# Flatten & train/test split
ns = NX * NY * NZ
X_flat = X.reshape(ns, n_features)
Y_flat = Y.reshape(ns)

X_train, X_test, y_train, y_test = train_test_split(
X_flat, Y_flat, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled.shape, y_train.shape

((3200, 27), (3200,))

In [44]:
np.save(PROCESSED / 'X.npy', X)
np.save(PROCESSED / 'y.npy', Y)
np.save(PROCESSED / 'X_train_scaled.npy', X_train_scaled)
np.save(PROCESSED / 'X_test_scaled.npy', X_test_scaled)
np.save(PROCESSED / 'y_train.npy', y_train)
np.save(PROCESSED / 'y_test.npy', y_test)


print("Saved all processed arrays.")

Saved all processed arrays.
